# Tour of Flanders 2017
In this edition of the series looking at Strava leaderboards LINK!!!, we turn to the action in Belgium. Greg Van Avermaet, Philipe Gilbert, Michal Kwiatkowski and Peter Sagan are among the riders in top form ahead of the Tour of Flanders, not forgetting former winners Tom Boonen, Alexander Kristoff and Stijn Devolder. This year's race LINK!!! includes 18 climbs, finishing with a loop that takes in six famous ascents in the last 50km. Should we expect to the pros to be setting KOMs on these hellingen bergs?<br>
The following table provides a summary of the Strava segments that correspond to the last six climbs. They are generally short and steep.

Hitting the top of any leaderboard towards the end of a 261km race sounds like a tall order. In fact they will be climbing the Oude-Kwaremont three times and the Paterberg twice, so the KOMs are more at risk on those two, if a rider decides to make a strong attack. The weather forecast is good: sunny, about 16C, with a light breeze from WNW. The wind will be against the riders on the Kruisberg, but it will provide a small benefit on the other final hills, which happen to be ridden in directions between East and South. 


In [27]:
import pandas as pd
import numpy as np

import Strava_Download as sd 

import stravalib
f = open( 'mytoken.txt', 'r' )
mytoken = f.read()
f.close()
client = stravalib.Client(access_token = mytoken)


In [28]:
tofSegIDs = [13593953, 642009, 661261, 640943, 608879, 640400]
surface = ['Asphalt','Cobbles','Cobbles','Asphalt','Cobbles','Cobbles']
segments = {}

for s in tofSegIDs:
    seg = client.get_segment(s)
    segments[s] = seg
    

# Strava segment attributes
class stravalib.model.Segment(bind_client=None, **kwargs)[source]
Bases: stravalib.model.LoadableEntity

Represents a single Strava segment.

name
Name of the segment.

activity_type
Activity type of segment (‘Ride’ or ‘Run’)

distance
Distance of segment

average_grade
Average grade (%) for segment

maximum_grade
Maximum grade (%) for segment

elevation_high
The highest point of the segment.

elevation_low
The lowest point of the segment.

start_latlng
The start lat/lon (tuple)

end_latlng
The end lat/lon (tuple)

start_latitude
The start latitude (float)

end_latitude
The end latitude (float)

start_longitude
The start longitude (float)

end_longitude
The end longitude (float)

city
The city this segment is in.

state
The state this segment is in.

country
The country this segment is in.

private
Whether this is a private segment.

starred
Whether this segment is starred by authenticated athlete

athlete_segment_stats
Undocumented attrib holding stats for current athlete.

created_at
datetime.datetime when was segment created.

updated_at
datetime.datetime when was segment last updated.

total_elevation_gain
What is total elevation gain for segment.

map
stravalib.model.Map object for segment.

effort_count
How many times has this segment been ridden.

athlete_count
How many athletes have ridden this segment

hazardous
Whether this segment has been flagged as hazardous

star_count
number of stars on this segment.

leaderboard
The stravalib.model.SegmentLeaderboard object for this segment.

In [29]:
segmentAttributes = ['name','activity_type','distance','average_grade','maximum_grade','elevation_high','elevation_low',
              'start_latlng','end_latlng','start_latitude','end_latitude','start_longitude','end_longitude',
              'city','state','country','private','starred','athlete_segment_stats','created_at','updated_at',
              'total_elevation_gain','map','effort_count','athlete_count','hazardous','star_count','leaderboard']
frameValues = ['name','average_grade','maximum_grade',
              'start_latitude','end_latitude','start_longitude','end_longitude',
              'effort_count','athlete_count']
frameQuantities = ['distance','elevation_high','elevation_low',
              'total_elevation_gain']


In [30]:
dfSegments = pd.DataFrame()
for s in tofSegIDs:
    dfSegments[s] = [getattr(segments[s],v) for v in frameValues ] + [float(stravalib.unithelper.meters(getattr(segments[s],q))) for q in frameQuantities ]
dfSegments = dfSegments.T
dfSegments.columns = [i.title() for i in frameValues + frameQuantities]
dfSegments['Surface'] = surface
dfSegments['Bearing'] = dfSegments.apply(lambda row:  int(180 + 180 / np.pi * np.arctan((row['End_Longitude']-row['Start_Longitude'])/(row['End_Latitude']-row['Start_Latitude']))), axis=1)
dfSegments['Total_Elevation_Gain'] = dfSegments['Elevation_High'] - dfSegments['Elevation_Low'] 
dfSegments[['Name','Surface', 'Distance', 'Average_Grade','Total_Elevation_Gain','Bearing']]

,Name,Surface,Distance,Average_Grade,Total_Elevation_Gain,Bearing
13593953,Koppenberg,Asphalt,444.2,14.3,63.6,104
642009,Steenbeekdries,Cobbles,724.1,2.8,24,121
661261,Taaienberg,Cobbles,638.64,7.8,45.7,262
640943,Kruisberg,Asphalt,1813.2,4.9,89,142
608879,Oude Kwaremont,Cobbles,2509.38,3.6,90.6,163
640400,Paterberg,Cobbles,358.01,11.7,42.2,96


In [31]:
varList = ['rank', 'effort_id', 'athlete_id', 'athlete_name',
           'athlete_gender','athlete_profile', 'average_hr',
           'average_watts', 'distance', 'elapsed_time',                                
           'moving_time', 'start_date_local', 'activity_id']

dfLeaderboards = pd.DataFrame()
for s in tofSegIDs:
    lb = segments[s].leaderboard
    lbDict = {}
    for var in varList:
        lbDict[var] = [getattr(e,var) for e in lb.entries]
    df = pd.DataFrame(lbDict)
    df.columns = [c.title() for c in df.columns]
    df['Segment'] = s
    df['Segment_Name'] = dfSegments.Name.loc[s]
    dfLeaderboards = dfLeaderboards.append(df)

dfLeaderboards
dfLeaderboards.index = range(len(dfLeaderboards))    

    

In [32]:
historicTofs = pd.to_datetime(pd.Series(['3-Apr-2016','5-Apr-2015','6-Apr-14','31-Mar-13','1-Apr-12','3-Apr-2011'])).dt.date
historicE3s = pd.to_datetime(pd.Series(['24-Mar-2017','25-Mar-2016','27-Mar-2015','28-Mar-14','22-Mar-13','23-Mar-12','26-Mar-2011'])).dt.date
historic3Days = pd.to_datetime(pd.Series(['27-Mar-2017','28-Mar-2017','29-Mar-2016','30-Mar-2016','31-Mar-2016','31-Mar-2015','1-Apr-2015','2-Apr-15','1-Apr-14','2-Apr-14','3-Apr-14','26-Mar-13','27-Mar-13','28-Mar-13','27-Mar-12','28-Mar-12','29-Mar-12','29-Mar-2011','30-Mar-2011','31-Mar-2011'])).dt.date
historicDDVs = pd.to_datetime(pd.Series(['22-Mar-2017','23-Mar-2016','25-Mar-2015','26-Mar-14','20-Mar-13','21-Mar-12','23-Mar-2011'])).dt.date


In [33]:
dfLeaderboards['Date'] = dfLeaderboards.Start_Date_Local.dt.date
dfLeaderboards['Race'] = '-'

In [34]:
dfLeaderboards.loc[dfLeaderboards.Date.isin(historicTofs.values),'Race'] = 'Tour of Flanders'
dfLeaderboards.loc[dfLeaderboards.Date.isin(historicE3s.values),'Race'] = 'E3 Harelbeke'
dfLeaderboards.loc[dfLeaderboards.Date.isin(historic3Days.values),'Race'] = 'Three Days of De Panne'
dfLeaderboards.loc[dfLeaderboards.Date.isin(historicDDVs.values),'Race'] = 'Dwars door Vlaanderen'
dfLeaderboards[['Segment_Name','Rank','Athlete_Name','Elapsed_Time','Date','Race']]

,Segment_Name,Rank,Athlete_Name,Elapsed_Time,Date,Race
0,Koppenberg,1,Reinardt Janse van Rensburg,00:01:27,2014-02-19,-
1,Koppenberg,2,Joris Van Der Auwera,00:01:28,2010-01-01,-
2,Koppenberg,2,gijsade holstege,00:01:28,2010-01-01,-
3,Koppenberg,2,Dries Devenyns,00:01:28,2015-11-27,-
4,Koppenberg,5,Cameron Bayly,00:01:30,2015-09-06,-
5,Koppenberg,6,Dylan Kennett,00:01:32,2015-06-26,-
6,Koppenberg,7,FOCUS Rides,00:01:34,2010-01-01,-
7,Koppenberg,7,Korneel De Viaene,00:01:34,2015-08-06,-
8,Koppenberg,7,Arjen Palstra,00:01:34,2016-04-02,-
9,Koppenberg,7,Wim Van Bever,00:01:34,2017-03-15,-


In [35]:
grouped =  dfLeaderboards.groupby('Segment_Name')

for name, group in grouped:
    print(name,'\n', group[['Rank','Athlete_Name','Start_Date_Local','Race']])

Koppenberg 
    Rank                 Athlete_Name    Start_Date_Local Race
0     1  Reinardt Janse van Rensburg 2014-02-19 12:47:29    -
1     2         Joris Van Der Auwera 2010-01-01 03:32:32    -
2     2             gijsade holstege 2010-01-01 14:49:26    -
3     2               Dries Devenyns 2015-11-27 11:27:43    -
4     5                Cameron Bayly 2015-09-06 15:33:55    -
5     6                Dylan Kennett 2015-06-26 12:26:56    -
6     7                  FOCUS Rides 2010-01-01 06:24:18    -
7     7            Korneel De Viaene 2015-08-06 11:33:56    -
8     7                Arjen Palstra 2016-04-02 11:14:33    -
9     7                Wim Van Bever 2017-03-15 17:07:10    -
Kruisberg 
     Rank       Athlete_Name    Start_Date_Local              Race
30     1       Daniel Lloyd 2011-04-03 13:20:57  Tour of Flanders
31     2             Cor ~~ 2011-04-03 13:33:04  Tour of Flanders
32     3         davide COM 2011-04-03 13:33:13  Tour of Flanders
33     4  Jeremy  Cameron Ⓥ 2

In [36]:
grouped.groups

{'Koppenberg': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'Kruisberg': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 'Oude Kwaremont': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 'Paterberg': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 'Steenbeekdries': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 'Taaienberg': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]}